In [ ]:
%run ../00_AdvancedPythonConcepts/talktools.py

<img src="https://www.evernote.com/l/AUV1r1xvhBdPF6lX-2SJLkO-vkkmCXEDrMwB/image.png">
http://www.slideshare.net/ManojitNandi/parallel-programming-in-python-speeding-up-your-analysis

Remember, you can create (fork) many processes, which are copies of the original parent process (memory, data, state) and act independently of each other. To share data between them you have to explicitly do that within each process. The Pythonic way we do multiprocessing (creation of new processes, communication between processes) is with `multiprocessing`.

*"effectively side-stepping the Global Interpreter Lock by using subprocesses instead of threads. Due to this, the multiprocessing module allows the programmer to fully leverage multiple processors on a given machine. It runs on both Unix and Windows."*

- https://docs.python.org/3/library/multiprocessing.html

The analog to `threading.Thread` is `multiprocessing.Process`. You should be able to do a drop-in replacement. Instead of `current_thread()` you'd use `os.getpid()`.

In [1]:
%load_ext snakeviz

In [3]:
%%snakeviz

import logging
import random
import time
import os

root = logging.getLogger()
root.handlers = []
logging.basicConfig(level=logging.DEBUG,
                    format='(%(threadName)-9s) %(message)s',)

import multiprocessing

def worker(num):
    """thread worker function"""
    
    sleep_time = random.randint(1,5)
    logging.debug('worker: {0} sleeping for {1} s, name: {2}'
                   .format(num,sleep_time,os.getpid()))
    time.sleep(sleep_time)
    logging.debug('done')
    return

procs = []
for i in range(2):
    p = multiprocessing.Process(target=worker, args=(i,))
    procs.append(p)
    p.start()

(MainThread) worker: 0 sleeping for 3 s, name: 1464
(MainThread) worker: 1 sleeping for 4 s, name: 1465


 
*** Profile stats marshalled to file '/var/folders/dc/1sk9z_cx15s7g6895fqjlp7h0000gn/T/tmp3zf50da1'. 


(MainThread) done
(MainThread) done


If your machine has multiple cores, these two processes may get run on those two separate cores, independently.

You may need to share info between processes. You can do this, just like with Threads with `Queues`. You can also use the (UNIX-like) Pipe to have  two processes communicate with each other:

In [4]:
# https://docs.python.org/3/library/multiprocessing.html#exchanging-objects-between-processes
from multiprocessing import Process, Pipe

def f(conn):
    conn.send([42, None, 'hello'])
    conn.close()

if __name__ == '__main__':
    parent_conn, child_conn = Pipe()
    p = Process(target=f, args=(child_conn,))
    p.start()
    print(parent_conn.recv())   # prints "[42, None, 'hello']"
    p.join()

[42, None, 'hello']


Using pools of workers (in separate processes) with multiprocessing Pool.

https://docs.python.org/3.5/library/multiprocessing.html#using-a-pool-of-workers

In [5]:
from multiprocessing import Pool 
pool = Pool(processes=4)     # start 4 worker processes 

In [6]:
pool

In [7]:
import time
def g(x):
    # domain specific stuff here!
    time.sleep(0.2)
    return x*x

In [8]:
%time list(map(g,range(10)))

CPU times: user 2.57 ms, sys: 1.49 ms, total: 4.06 ms
Wall time: 2.03 s


[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [11]:
%time pool.map(g,range(10))

CPU times: user 2.57 ms, sys: 2.54 ms, total: 5.11 ms
Wall time: 605 ms


[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [13]:
# print same numbers in arbitrary order
for i in pool.imap_unordered(g, range(10)):
    print(i,sep=" ",end=" ")

0 4 1 9 25 36 16 49 64 81 

Python is packaging (pickling) up your functions and sending them to different processes.

In [14]:
pool.map(lambda x: x**3, range(10))

PicklingError: Can't pickle <function <lambda> at 0x104c81488>: attribute lookup <lambda> on __main__ failed

In [19]:
# run only one process "g(10)" asynchronously 
result = pool.apply_async(g, [10])

# prints "100" unless you timeout
print(result.get(timeout=0.25)) 

100


In [16]:
result

In [20]:
del pool

In [23]:
from multiprocessing import Pool 
import time

def f(x): 
    return x*x

for i in [1,2,3,4,8,16,32]:
    print(i,"*"*5,flush=True)
    pool = Pool(processes=i)               # start 4 worker processes 
    start = time.time()
    pool.map(f, range(1000000))
    print("{0:0.4f} sec".format(time.time() - start))
    pool.terminate()
    del pool

1 *****
0.5565 sec
2 *****
0.3939 sec
3 *****
0.3765 sec
4 *****
0.3558 sec
8 *****
0.4504 sec
16 *****
0.3872 sec
32 *****
0.4038 sec


In [24]:
!ulimit -a

core file size          (blocks, -c) 0
data seg size           (kbytes, -d) unlimited
file size               (blocks, -f) unlimited
max locked memory       (kbytes, -l) unlimited
max memory size         (kbytes, -m) unlimited
open files                      (-n) 256
pipe size            (512 bytes, -p) 1
stack size              (kbytes, -s) 8192
cpu time               (seconds, -t) unlimited
max user processes              (-u) 709
virtual memory          (kbytes, -v) unlimited


# `concurrent.futures` - Launching parallel tasks

Built-in, create different pools for executing **maps** (single loop over data). Local resources.

`The concurrent.futures module provides a high-level interface for asynchronously executing callables.

The asynchronous execution can be performed with threads, using ThreadPoolExecutor, or separate processes, using ProcessPoolExecutor. Both implement the same interface, which is defined by the abstract Executor class.`

https://docs.python.org/3/library/concurrent.futures.html

In [31]:
from concurrent.futures import ProcessPoolExecutor
e = ProcessPoolExecutor(2)  # can also use a threadpool

In [27]:
%%time 
from time import sleep

results = []
for i in range(8):
    sleep(1)
    results.append(i + 1)

CPU times: user 8.14 ms, sys: 2.9 ms, total: 11 ms
Wall time: 8.02 s


In [28]:
results

[1, 2, 3, 4, 5, 6, 7, 8]

In [33]:
%%time 
from time import sleep

from concurrent.futures import ProcessPoolExecutor
e = ProcessPoolExecutor(2) 

def slowfunc(x):
    sleep(1)
    return(x+1)

results = list(e.map(slowfunc,range(8)))

CPU times: user 12.4 ms, sys: 11.6 ms, total: 24 ms
Wall time: 4.02 s


In [32]:
e.shutdown()

Figured out I have 4 cores and ran it in 4 separate processes.

## Breakout

Convert the sequential code to parallel using `concurrent.futures`

In [ ]:
%%time

import requests
from bs4 import BeautifulSoup

url = "https://en.wikipedia.org/wiki/Special:Random"

lens = []
for i in range(10):
    a = requests.get(url)
    resp = a.text
    print("title=",BeautifulSoup(resp, 'html.parser')
          .title.string.split("- Wikipedia")[0],"len=",len(resp))
    lens.append(len(resp))

print(lens)

### Executor.submit

`submit` starts an execution in a separate thread or process and immediately returns a `Future` object that points back to the result. Until the function completes, the future is pending. We get the result of a task with `.result()`, which blocks until the computation is complete.

In [34]:
%%time 
from time import sleep

from concurrent.futures import ProcessPoolExecutor
e = ProcessPoolExecutor() 

def slowfunc(x,y,delay=1):
    sleep(delay)
    return(x+y)

future = e.submit(slowfunc,1, 2)

CPU times: user 6.52 ms, sys: 16.4 ms, total: 22.9 ms
Wall time: 21.1 ms


In [35]:
future

<Future at 0x104cb2940 state=finished returned int>

In [36]:
future.result()

3

In [37]:
%%time 
futures = [e.submit(slowfunc,1,2, delay=1) for _ in range(10)]
results = [f.result() for f in futures]

CPU times: user 9.05 ms, sys: 4.02 ms, total: 13.1 ms
Wall time: 3.01 s


## Joblib

http://pythonhosted.org/joblib/

Running Python functions as pipeline jobs. The *vision is to provide tools to easily achieve better performance and reproducibility when working with long running jobs.* Specifically meant to work well with large data (ie. numpy arrays).

  - **Avoid computing twice the same thing**: code is rerun over an over, for instance when prototyping computational-heavy jobs (as in scientific development), but hand-crafted solution to alleviate this issue is error-prone and often leads to unreproducible results
  - **Persist to disk transparently**: persisting in an efficient way arbitrary objects containing large data is hard. Using joblib’s caching mechanism avoids hand-written persistence and implicitly links the file on disk to the execution context of the original Python object. As a result, joblib’s persistence is good for resuming an application status or computational job, eg after a crash.

Joblib strives to address these problems while leaving your code and your flow control as unmodified as possible (no framework, no new paradigms).

In [ ]:
!conda install joblib -y

In [38]:
from math import sqrt
[sqrt(i ** 2) for i in range(10)]

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]

### Parallel Helpers

Joblib provides a simple helper class to write parallel for loops using multiprocessing. The core idea is to write the code to be executed as a generator expression, and convert it to parallel computing.

In [39]:
from math import sqrt
from joblib import Parallel, delayed

By default Parallel uses the Python multiprocessing module to fork separate Python worker processes to execute tasks concurrently on separate CPUs. This is a reasonable default for generic Python programs but it induces some overhead as the input and output data need to be serialized in a queue for communication with the worker processes. 

In [40]:
Parallel(n_jobs=2,backend="threading") \
  (delayed(sqrt)(i ** 2) for i in range(10))

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]

In [44]:
import time
start = time.time()
Parallel(n_jobs=5,verbose=5) \
  (delayed(time.sleep)(1) for _ in range(100))
print(time.time()-start)

[Parallel(n_jobs=5)]: Done   8 tasks      | elapsed:    2.0s
[Parallel(n_jobs=5)]: Done  62 tasks      | elapsed:   13.0s


20.16140079498291


[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:   20.0s finished


### On demand recomputing: the `Memory` class

Caching long running results so it can be reused. Let's try to cache to disk:

In [50]:
from joblib import Memory
memory = Memory(cachedir="/tmp/", verbose=5)

In [51]:
@memory.cache
def f(x):
    print('Running f(%s)' % x)
    return x

In [52]:
print(f(1))

[Memory]    2.4s, 0.0min: Loading f...
___________________________________________________f cache loaded - 0.0s, 0.0min
1


In [53]:
print(f(1))

[Memory]    8.8s, 0.1min: Loading f...
___________________________________________________f cache loaded - 0.0s, 0.0min
1


In [54]:
print(f(10))

[Memory]   18.1s, 0.3min: Loading f...
___________________________________________________f cache loaded - 0.0s, 0.0min
10


In [58]:
print(f(20))

________________________________________________________________________________
[Memory] Calling __main__--Users-jbloom-Classes-python-seminar-DataFiles_and_Notebooks-08_Parallelism-__ipython-input__.f...
f(20)
Running f(20)
________________________________________________________________f - 0.0s, 0.0min
20


In [59]:
!ls -lat /tmp/joblib/__main__--Users-jbloom-Classes-python-seminar-DataFiles_and_Notebooks-08_Parallelism-__ipython-input__/f

total 8
drwxr-xr-x  6 jbloom  wheel  204 Sep 23 15:14 .
drwxr-xr-x  4 jbloom  wheel  136 Sep 23 15:14 384f3b7270f5bd1b16f875a18665c26e
drwxr-xr-x  5 jbloom  wheel  170 Sep 23 11:42 ..
drwxr-xr-x  4 jbloom  wheel  136 Sep 23 11:41 1d57c12eb7e5ebcc7910478490915e9a
drwxr-xr-x  4 jbloom  wheel  136 Sep 23 11:41 d3ffa92536e9b2aeb96c6d0e11ccd857
-rw-r--r--  1 jbloom  wheel   84 Sep 23 11:41 func_code.py


In [68]:
memory = Memory(cachedir="/tmp/",verbose=10,mmap_mode="r+")

In [72]:
@memory.cache
def josh(x,blah=True):
    print('Running g(%s)' % x)
    return x

In [73]:
print(josh(1))

________________________________________________________________________________
[Memory] Calling __main__--Users-jbloom-Classes-python-seminar-DataFiles_and_Notebooks-08_Parallelism-__ipython-input__.josh...
josh(1)
Running g(1)
_____________________________________________________________josh - 0.0s, 0.0min
[Memory]   17.0s, 0.3min: Loading josh from /tmp/joblib/__main__--Users-jbloom-Classes-python-seminar-DataFiles_and_Notebooks-08_Parallelism-__ipython-input__/josh/5af310b98119c5e5b1a775c624b66fb1
1


In [74]:
print(josh(1))

[Memory]   26.0s, 0.4min: Loading josh from /tmp/joblib/__main__--Users-jbloom-Classes-python-seminar-DataFiles_and_Notebooks-08_Parallelism-__ipython-input__/josh/5af310b98119c5e5b1a775c624b66fb1
________________________________________________josh cache loaded - 0.0s, 0.0min
1


In [64]:
print(g(1,blah=False))

1


Ignoring variables:

In [65]:
@memory.cache(ignore=['blah'])
def h(x,blah=True):
    print('Running h(%s)' % x)
    return x

In [66]:
print(h(1))

1


In [67]:
print(h(1,blah=False))

1


Note: joblib also gives (for persistence) `joblib.dump()` and `joblib.load()` provide a replacement for pickle to work efficiently on Python objects containing large data, in particular large numpy arrays.